In [1]:
import pandas as pd
from datetime import datetime

# Load the first Excel file: faults data
faults_df = pd.read_excel("Telegeography data.xlsx", usecols=[0], header=0)
faults_df.columns = ["name"]

# Standardize names: strip and lowercase
faults_df["name"] = faults_df["name"].str.strip().str.lower()

# Count number of faults per cable
fault_counts = faults_df["name"].value_counts().reset_index()
fault_counts.columns = ["name", "number_of_faults"]

# Load the second Excel file: submarine cable data
submarine_file = "Telegeography_submarine cable data.xlsx"

# Sheet: submarine cable networks (age)
networks_df = pd.read_excel(
    submarine_file,
    sheet_name="submarine cable networks",
    usecols=[0, 16, 17],
    header=0
)
networks_df.columns = ["name", "year", "month"]

# Standardize names
networks_df["name"] = networks_df["name"].str.strip().str.lower()

# Clean year column: convert to numeric, force errors to NaN
networks_df["start_year"] = pd.to_numeric(networks_df["year"], errors="coerce")

# Calculate age only where start_year is valid
current_year = datetime.now().year
networks_df["age"] = networks_df["start_year"].apply(
    lambda y: current_year - int(y) if pd.notna(y) else None
)

networks_df = networks_df[["name", "age"]]

# Sheet: submarine cable capacity (capacity)
capacity_df = pd.read_excel(
    submarine_file,
    sheet_name="submarine cable capacity",
    usecols=[0, 6],
    header=0
)
capacity_df.columns = ["name", "capacity"]

# Standardize names
capacity_df["name"] = capacity_df["name"].str.strip().str.lower()

# Merge everything together
merged_df = pd.merge(fault_counts, networks_df, on="name", how="outer")
merged_df = pd.merge(merged_df, capacity_df, on="name", how="outer")

# Replace NaN fault counts with 0
merged_df["number_of_faults"] = merged_df["number_of_faults"].fillna(0).astype(int)

# Save to new Excel file
merged_df.to_excel("data.xlsx", index=False)

In [2]:
import pandas as pd
import numpy as np
import re

# load
df = pd.read_excel("data.xlsx")

# standardize names
df['name'] = df['name'].astype(str).str.strip().str.lower()

# drop original_count if present
if 'original_count' in df.columns:
    df = df.drop(columns=['original_count'])

# ensure numeric faults and age
df['number_of_faults'] = pd.to_numeric(df.get('number_of_faults', 0), errors='coerce').fillna(0).astype(int)
df['age'] = pd.to_numeric(df.get('age'), errors='coerce')

# helper to extract first numeric token from capacity (handles commas, decimals)
def extract_number(val):
    if pd.isna(val):
        return np.nan
    s = str(val).replace(',', '').strip()
    m = re.search(r'[-+]?\d*\.?\d+', s)
    return float(m.group(0)) if m else np.nan

# create numeric capacity column (if capacity column exists)
if 'capacity' in df.columns:
    df['capacity_num'] = df['capacity'].apply(extract_number)
else:
    df['capacity_num'] = np.nan

# group — do NOT sum faults; take max (keeps correct value if duplicates are identical)
grouped = df.groupby('name', as_index=False).agg({
    'number_of_faults': 'max',
    'age': lambda s: s.dropna().min() if s.dropna().size > 0 else np.nan,
    'capacity_num': lambda s: s.dropna().min() if s.dropna().size > 0 else np.nan
})

# format capacity: integer if whole number, else float; None if nan
def fmt_capacity(x):
    if pd.isna(x):
        return None
    if float(x).is_integer():
        return int(x)
    return float(x)

grouped['capacity'] = grouped['capacity_num'].apply(fmt_capacity)
grouped = grouped.drop(columns=['capacity_num'])

# cast age to nullable integer if possible
grouped['age'] = grouped['age'].astype('Int64')

# ensure number_of_faults is int
grouped['number_of_faults'] = grouped['number_of_faults'].astype(int)

# save back to data.xlsx (overwrite)
grouped.to_excel("data.xlsx", index=False)

print(f"Deduplicated saved to data.xlsx — rows before: {len(df)}, rows after: {len(grouped)}")


Deduplicated saved to data.xlsx — rows before: 776, rows after: 664


In [3]:
import pandas as pd
import numpy as np

# Load cleaned data
df = pd.read_excel("data.xlsx")

# Ensure numeric types
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['number_of_faults'] = pd.to_numeric(df['number_of_faults'], errors='coerce').fillna(0).astype(int)
df['capacity'] = pd.to_numeric(df['capacity'], errors='coerce')

# Formula: vul_score = age + faults^1.1 + (0 * capacity)
def calc_vul_score(row):
    if pd.isna(row['age']) or row['age'] < 0:
        return 0
    score = (np.log(row['age'])) + (row['number_of_faults'] ** 1.4) + (0 * (row['capacity'] if not pd.isna(row['capacity']) else 0))
    return score

df['vul_score'] = df.apply(calc_vul_score, axis=1)

# Rank: higher score = lower rank (rank 1 = most vulnerable)
df['vul_rank'] = df['vul_score'].rank(method='dense', ascending=False).astype(int)

# Reorder columns: name, vul_rank, vul_score, number_of_faults, age, capacity
final = df[['name', 'vul_rank', 'vul_score', 'number_of_faults', 'age', 'capacity']]

# Sort by rank
final = final.sort_values('vul_rank')

# Save to new file
final.to_excel("vul.xlsx", index=False)

print("vul.xlsx created successfully!")

# Display the formula
print("Formula used: vul_score = log(age) + (number_of_faults ^ 1.4) + (0 * capacity)")


vul.xlsx created successfully!
Formula used: vul_score = log(age) + (number_of_faults ^ 1.4) + (0 * capacity)


C:\Users\aadit\AppData\Local\Temp\ipykernel_18304\3296694967.py:16: RuntimeWarning: divide by zero encountered in log
  score = (np.log(row['age'])) + (row['number_of_faults'] ** 1.4) + (0 * (row['capacity'] if not pd.isna(row['capacity']) else 0))
